In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from pathlib import Path

In [ ]:
# Login to Hugging Face Hub
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

In [ ]:
# Define your Hugging Face Hub model repo ID
hub_model_id = os.getenv("HUB_MODEL_ID")

In [ ]:
# Load dataset
project_root = Path().cwd().resolve().parent
dataset_dir = project_root / "data" / "processed"
dataset = load_dataset('csv', data_files={'train': str(dataset_dir / 'train.csv'), 'val': str(dataset_dir / 'val.csv'), 'test': str(dataset_dir / 'test.csv')})
dataset

In [ ]:
# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples['user_utterance'], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Model Setup
num_labels = len(dataset['train'].unique('label'))
id2label = {i: label for i, label in enumerate(sorted(set(dataset['train']['new_intent'])))}
label2id = {label: i for i, label in id2label.items()}
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
def compute_validation_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return {"val_accuracy": acc, "val_f1": f1}

In [ ]:
# Set up Training Arguments with push_to_hub enabled
training_args = TrainingArguments(
    output_dir=str(project_root / "models" / "intent_classifier"),
	eval_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-6,
    weight_decay=1e-3,
    num_train_epochs=2,
    logging_strategy="epoch",
    save_strategy="best",
    disable_tqdm=False,
    load_best_model_at_end=True,
    # --- Hugging Face Hub arguments ---
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
    hub_token=hf_token,
    report_to="none"
)

In [ ]:
# Create and run the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_validation_metrics,
)

trainer.train()

In [ ]:
# Get predictions on the test set
test_predictions = trainer.predict(tokenized_datasets["test"])

# Extract the predicted labels
y_pred = np.argmax(test_predictions.predictions, axis=1)
y_true = test_predictions.label_ids

In [ ]:
# Generate and print the Classification Report
target_names = sorted(set(dataset['test']['new_intent']))
report = classification_report(y_true, y_pred, target_names=target_names)
print("\nClassification Report:")
print(report)

In [ ]:
# Generate and save the Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
trainer.push_to_hub(commit_message="DistilBert fine-tuned intent classifier")